# Merging data tables in Pandas

We would like to take two data files with similar data and merge them into one data table.  I have two Gamma-ray burst files here with some data.  Some of the columns in both of the files match, which is what we'd expect.

One file has two additional Gamma-ray bursts (GRBs) that the other does not.  We only want to keep the matching GRBs, but I'll show you how to keep all.

---

#### Read in data that is separated into columns within the file by 1 space.
#### Data has a row of headers.

In [39]:
import pandas

In [40]:
f1 = "/Users/KimiZ/GRBs2/analysis/Sample/bursts.txt"
f2 = "/Users/KimiZ/GRBs2/analysis/Sample/bursts_LATboresight_angles.txt"

data1 = pandas.read_table(f1, delimiter=" ", header=0)
data2 = pandas.read_table(f2, delimiter=" ", header=0)

In [41]:
data1[0:5]

,number,name,trigger,z,t90_start,t90_stop
0,1,GRB080804,bn080804972,2.2045,0.256,24.960
1,2,GRB080810,bn080810549,3.3604,-20.096,87.361
2,3,GRB080905A,bn080905499,0.1218,-0.064,0.896
3,4,GRB080905B,bn080905705,2.3739,-5.120,100.864
4,5,GRB080916A,bn080916406,0.6887,0.512,46.849


In [42]:
data2[0:5]

,name,trigger,z,BATdet,LATdet,angT0,angT30,angT60
0,GRB080804,bn080804972,2.2045,yes,no,0.0000,0.0000,0.0000
1,GRB080810,bn080810549,3.3604,yes,no,0.0000,0.0000,0.0000
2,GRB080905A,bn080905499,0.1218,yes,no,27.8855,29.1496,30.4518
3,GRB080905B,bn080905705,2.3739,yes,no,85.4289,85.8448,86.2924
4,GRB080916A,bn080916406,0.6887,yes,no,75.8313,77.1288,78.4168


### Lets get rid of a few columns we don't care about.
Anything matching (except 1 matching column) and any columns in either data frame that we don't care about.

#### ** We want to keep one column in the data frames that match so that we have a way to tell Pandas how to merge the two data frames.  'trigger' will be that column.

If the column names do not match in the data files, you could change them by hand in the files, or do them now:




In [43]:
cols = data1.columns

In [44]:
cols

Index([u'number', u'name', u'trigger', u'z', u't90_start', u't90_stop'], dtype='object')

In [45]:
# data1.columns = "a,b,c,d,e,f".split(',')
# or 
data1.columns = ["a","b","c","d","e","f"]

In [46]:
data1[0:2]

,a,b,c,d,e,f
0,1,GRB080804,bn080804972,2.2045,0.256,24.960
1,2,GRB080810,bn080810549,3.3604,-20.096,87.361


#### Change them back:

In [47]:
data1.columns = cols

In [48]:
data1[0:2]

,number,name,trigger,z,t90_start,t90_stop
0,1,GRB080804,bn080804972,2.2045,0.256,24.960
1,2,GRB080810,bn080810549,3.3604,-20.096,87.361


### Okay, back to dropping columns:

    axis = 1         # 0-rows, 1-columns
    inplace = True   # replace the data frame with an adjusted data frame.
                     # inplace=False is the default
                     
inplace=False is the default, which means that in order to use the adjusted data frame with the dropped columns, you'd have to assign it to a new data frame:

    data1_new = data1.drop(labels=['number'], axis=1)
    
If we use inplace=True, data1 is automatically replaced.

In [49]:
data1.drop(labels=['number'], axis=1, inplace=True)

In [50]:
data2.drop(labels=['name', 'z'], axis=1, inplace=True)

In [51]:
data1[0:5]

,name,trigger,z,t90_start,t90_stop
0,GRB080804,bn080804972,2.2045,0.256,24.960
1,GRB080810,bn080810549,3.3604,-20.096,87.361
2,GRB080905A,bn080905499,0.1218,-0.064,0.896
3,GRB080905B,bn080905705,2.3739,-5.120,100.864
4,GRB080916A,bn080916406,0.6887,0.512,46.849


In [52]:
data2[0:5]

,trigger,BATdet,LATdet,angT0,angT30,angT60
0,bn080804972,yes,no,0.0000,0.0000,0.0000
1,bn080810549,yes,no,0.0000,0.0000,0.0000
2,bn080905499,yes,no,27.8855,29.1496,30.4518
3,bn080905705,yes,no,85.4289,85.8448,86.2924
4,bn080916406,yes,no,75.8313,77.1288,78.4168


## Now we merge

#### We want to merge the two data frames by the column named 'trigger'

In [53]:
data = pandas.merge(left=data1, right=data2, on='trigger')

In [54]:
data[0:5]

,name,trigger,z,t90_start,t90_stop,BATdet,LATdet,angT0,angT30,angT60
0,GRB080804,bn080804972,2.2045,0.256,24.960,yes,no,0.0000,0.0000,0.0000
1,GRB080810,bn080810549,3.3604,-20.096,87.361,yes,no,0.0000,0.0000,0.0000
2,GRB080905A,bn080905499,0.1218,-0.064,0.896,yes,no,27.8855,29.1496,30.4518
3,GRB080905B,bn080905705,2.3739,-5.120,100.864,yes,no,85.4289,85.8448,86.2924
4,GRB080916A,bn080916406,0.6887,0.512,46.849,yes,no,75.8313,77.1288,78.4168


In [59]:
data.to_csv('burst_data.txt', sep = ' ', header=True, index_label='number')

### \*** IMPORTANT NOTE:

You should notice that the sizes of data1 and data2 were different.  data2 had two extra Gamma-ray bursts (GRBs) than data1.  

After we merge the two data frames, the final data frame, data, has the same length as data1.

Pandas.merge only merges the *matching* data, and omits the data that doesn't match.  This is exactly what we wanted here.

In [17]:
len(data1), len(data2), len(data)

(70, 72, 70)

### What if we wanted those keep the 2 additional GRBs?

Using the how='outer' does the job.
outer: use union of keys from both frames (SQL: full outer join)
see:
    
    help(pandas.merge)


In [18]:
data_alt = pandas.merge(left=data1, right=data2, on='trigger', how='outer')

In [19]:
len(data_alt)

72

### The two extra GRBs will be at the end of the data frame.

In [20]:
data_alt

,name,trigger,z,t90_start,t90_stop,BATdet,LATdet,angT0,angT30,angT60
0,GRB080804,bn080804972,2.2045,0.256,24.960,yes,no,0.00000,0.00000,0.00000
1,GRB080810,bn080810549,3.3604,-20.096,87.361,yes,no,0.00000,0.00000,0.00000
2,GRB080905A,bn080905499,0.1218,-0.064,0.896,yes,no,27.88550,29.14960,30.45180
3,GRB080905B,bn080905705,2.3739,-5.120,100.864,yes,no,85.42890,85.84480,86.29240
4,GRB080916A,bn080916406,0.6887,0.512,46.849,yes,no,75.83130,77.12880,78.41680
5,GRB080916C,bn080916009,4.3500,1.280,64.257,no,yes,48.83960,48.65890,48.48500
6,GRB080928,bn080928628,1.6919,-1.792,12.544,yes,no,45.21530,44.34000,43.47110
7,GRB081008,bn081008832,1.9670,0.004,150.019,yes,no,64.26970,64.97110,65.66190
8,GRB081121,bn081121858,2.5120,1.536,43.521,yes,no,134.73900,133.96900,133.19500
9,GRB081221,bn081221681,2.2600,3.328,33.025,yes,no,77.21590,78.93890,80.74000


----

# Extra Stuff:

### Lets start with the original data frames 

In [21]:
data1 = pandas.read_table(f1, delimiter=" ", header=0)
data2 = pandas.read_table(f2, delimiter=" ", header=0)

In [22]:
len(data1), len(data2)

(70, 72)

data2 has two more Gamma-ray bursts (GRBs) in its table than data1.  We only want the GRBs that are in BOTH data frames.
#### Here is a trick 
to find the matching or non-matching bursts between data frames.

We make them both sets so that we can use the Python set feature difference.  This will give us the difference between the two sets.

    d2.difference(d1)
    

In [23]:
d1 = set(data1['trigger'])
d2 = set(data2['trigger'])

 
d2.difference(d1)

{'bn120118709', 'bn130907904'}

#### Here is another Example to you can see you this works:

In [24]:
D2 = {"cat","dog","horse","bird","fish"}
D1 = {"cat","dog","bear"}

#### What does d2 have that d1 does not:

In [25]:
D2.difference(D1)  

{'bird', 'fish', 'horse'}

#### What does d1 have that d2 does not:

In [26]:
D1.difference(D2)

{'bear'}

In [27]:
D2.intersection(D1)

{'cat', 'dog'}

In [28]:
D1.intersection(D2)

{'cat', 'dog'}

In [29]:
bursts_to_remove = d2.difference(d1)

It's important to put the shorter table (d1) as a subset of (d2) because d2 has all the GRBs that d1 does, but 2 extra.  So if you take the difference d1.difference(d2), nothing will be returned because there is NOTHING that d1 has that d2 does not.

We convert it back into a list in the same step because we like the indexing feature of lists.

In [30]:
bursts_to_remove = list(bursts_to_remove); bursts_to_remove

['bn130907904', 'bn120118709']

    data2[data2['trigger']=='bn120118709']
Returns the row of data in the data frame that belongs to the trigger name 'bn120118709'. 

    data2[data2['trigger']=='bn120118709'].index  

Returns:
    
    Int64Index([46], dtype='int64')

which appears to act an an array with the index in the [0] and dtype='int64' in the [1] spots.

So we'll use:
    
    data2[data2['trigger']=='bn120118709'].index[0]  
Returns:
    
    46


Again, because the drop feature defaults *inplace = False*, we either need to change it to True or set the call to a new data frame name.
If you are looping over a list, as we will, it is quickest (and cleanist) to use *inplace=True.*

In [31]:
for burst in bursts_to_remove:
    idx = data2[data2['trigger']== burst].index[0] 
    print burst, idx
    data2.drop(idx, inplace=True)

bn130907904 63
bn120118709 46


In [32]:
# Check that both data frames are the same lenght now.
len(data1) == len(data2)

True

In [33]:
data2[0:2]

,name,trigger,z,BATdet,LATdet,angT0,angT30,angT60
0,GRB080804,bn080804972,2.2045,yes,no,0.0,0.0,0.0
1,GRB080810,bn080810549,3.3604,yes,no,0.0,0.0,0.0


### Again, drop unnecessary columns

In [34]:
data1.drop(labels=['number'], axis=1, inplace=True)
data2.drop(labels=['z','name'], axis=1, inplace=True)

In [35]:
data1[0:2]

,name,trigger,z,t90_start,t90_stop
0,GRB080804,bn080804972,2.2045,0.256,24.960
1,GRB080810,bn080810549,3.3604,-20.096,87.361


In [36]:
data2[0:2]

,trigger,BATdet,LATdet,angT0,angT30,angT60
0,bn080804972,yes,no,0.0,0.0,0.0
1,bn080810549,yes,no,0.0,0.0,0.0


### Just to make sure the data in both data frames are in the same order:

I'm sure there are better ways to do this.  I just do what comes most natural for me.

In [37]:
d1_triggers = list(data1['trigger'])
d2_triggers = list(data2['trigger'])

for i,trig in enumerate(d1_triggers):
    print d2_triggers[i] == trig      # all should return True.



True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
